In [ ]:
import pandas as pd
# import modin.pandas as pd
import numpy as np
import pickle as pkl
from os import listdir, walk
from os.path import isfile, join
from dask.distributed import Client
import dask.dataframe as dd

import datetime

import json
import time

import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

%matplotlib inline

In [ ]:
# extracting admission source, chief complaint, and COPS2 commorbidity groupings

In [ ]:
cdm_s = pd.read_csv("/data/deidentified_trews_deployment_data/cdm/cdm_s.csv")

In [ ]:
conditions = set()
for x in sorted(cdm_s.fid.unique()):
    sstring_index1 = x.find('_final_diag')
    sstring_index2 = x.find('_diag')
    sstring_index3 = x.find('_hist')
    if sstring_index1 > 0:
        sstring = x[:sstring_index1]
    elif sstring_index2  > 0:
        sstring = x[:sstring_index2]
    elif sstring_index3 > 0:
        sstring = x[:sstring_index3]
    conditions.add(sstring)

In [ ]:
conditions

In [ ]:
sorted(cdm_s.fid.unique())

In [ ]:
aam_df = pd.read_hdf('/data/adarsh/fda_project_data/aam_model_dataset.h5')

In [ ]:
# just need the ids
aam_df = aam_df[['enc_id']].groupby('enc_id', as_index=False).nth(0)

In [ ]:
enc_ids = aam_df.enc_id.unique()

In [ ]:
# admit_sources = ['Home or Workplace or Non-Health Care Facility', "Physician's office or clinic", "Skilled Nursing Facility, Intermediate Care Facility or Assisted Living Facility"]
home_admits = cdm_s.loc[cdm_s.enc_id.isin(enc_ids)].query("fid == 'admit_source'").query('value == "Home or Workplace or Non-Health Care Facility"')[['enc_id']] # [['enc_id', 'value']].value.unique()
home_admits['admit_source'] = 1

clinic_string = "Physician's office or clinic"
clinic_admits = cdm_s.loc[cdm_s.enc_id.isin(enc_ids)].query("fid == 'admit_source'").query('value == @clinic_string')[['enc_id']] # [['enc_id', 'value']].value.unique() 
clinic_admits['admit_source'] = 2

# 0 is everything else

In [ ]:
aam_df = aam_df.merge(pd.concat([home_admits, clinic_admits]), on='enc_id', how='left')

In [ ]:
aam_df['admit_source'] = aam_df.admit_source.fillna(0)

In [ ]:
condition_df.shape[0]

In [ ]:
cdm_s = cdm_s.loc[cdm_s.enc_id.isin(enc_ids)]

In [ ]:
condition_df

In [ ]:
# now we get comorbidities
suffixes = ['_diag', '_final_diag', '_hist']
for condition in conditions:
    condition_names = [condition + suffix for suffix in suffixes]
    condition_df = cdm_s.query("fid in @condition_names").groupby('enc_id', as_index=False).nth(0)
    condition_df[condition] = 1
    aam_df = aam_df.merge(condition_df[['enc_id', condition]], on='enc_id', how='left')
    aam_df[condition] = aam_df[condition].fillna(0)
    print(condition, condition_df.shape[0])

In [ ]:
aam_df

In [ ]:
# TODO: get chief complaint

In [ ]:
aam_df

In [ ]:
aam_df.to_csv("/data/adarsh/fda_project_data/study_cohort_commorbidity_features.csv", index=False)

In [ ]:
cdm_s.query('fid == "chief_complaint"')

In [ ]:
wrong_ed_alert_ids = np.load("/data/adarsh/fda_project_data/predictions/ed_admits_wrong_alerts.npy")

In [ ]:
complaints = cdm_s.query('fid == "chief_complaint" and enc_id in @wrong_ed_alert_ids').value.values

In [ ]:
wrong_ed_complaint_df = cdm_s.query('fid == "chief_complaint" and enc_id in @wrong_ed_alert_ids')

In [ ]:
complaints

In [ ]:
json.loads(complaints[0])[0]['reason']

In [ ]:
unrolled_complaints = pd.json_normalize(cdm_s.query('fid == "chief_complaint" and enc_id in @wrong_ed_alert_ids').value.apply(json.loads).apply(lambda x: x[0]))

In [ ]:
wrong_ed_complaint_df['reason'] = unrolled_complaints['reason'].values
wrong_ed_complaint_df['comments'] = unrolled_complaints['comments'].values

In [ ]:
complaint_counts = wrong_ed_complaint_df[['enc_id', 'reason']].groupby('reason', as_index=False).count().sort_values('enc_id', ascending=False)

In [ ]:
complaint_counts.reason.values

In [ ]:
complaint_counts.query('enc_id > 10')